# Export survey data with attachments

<table class="tfo-notebook-buttons" align="right">
  <td>
    <a target="_blank" href="https://www.arcgis.com/home/item.html?id=b60a589edae249f293c3b880a33ee363">Try it live</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/Esri/Survey123-tools/tree/main/Export_survey_data_with_attachments">View source on GitHub</a>
  </td>
</table>

<div class="alert alert-block alert-info">This notebook uses the ArcGIS API for Python. For more information, see the <a href="https://developers.arcgis.com/python/">ArcGIS API for Python documentation and guides</a>.</div>

A common ArcGIS Survey123 workflow is to export survey data as a `.xlsx` file and work with that data in other software. A limitation of exporting data from your ArcGIS organization in Microsoft Excel format is that the attachments are not included. The only way to include attachments when exporting a dataset from the Survey123 website, ArcGIS Online, or ArcGIS Enterprise is to export it as a file geodatabase, but this method still requires additional work to extract the attachments.

Fortunately, you can use the ArcGIS API for Python to export your survey data in XLSX format and save any attachments to your computer.

This notebook will export your survey results as a `.xlsx` file, download all attachments associated with the data into folders unique to the parent layer or table, and create additional CSV files that map which parent object ID corresponds to which attachment file path.

In [1]:
import arcgis
from arcgis.gis import GIS
import os, re, csv

ModuleNotFoundError: No module named 'arcgis'

The first step is to define the variables needed to complete this workflow. The required variables are as follows: 
* **portalURL** - The URL for your ArcGIS organization (e.g. www.arcgis.com)
* **username** - Your ArcGIS organization username (e.g. gisadmin)
* **password** - You ArcGIS organization password (e.g. gisadmin1)
* **survey_item_id** - The item ID for the ArcGIS Survey123 form item in your ArcGIS organization (e.g. 89bc8c7844e548e09baa3aad4695e78b)
* **save_path** - The directory where you would like to save the survey results and attachments (e.g. C:\temp)
* **keep_org_item** - By default, an exported item is added to your content in your ArcGIS organization. This Boolean value allows you to choose if you would like to keep the exported item in your content (`True`), or remove it (`False`).
* **store_csv_w_attachments** - Boolean value that allows you to choose if the `.csv` file that maps each attachment to its parent object ID should be stored in the root folder (with the exported Excel workbook) (`False`), or in each individual layer folder (`True`).

In [2]:
portalURL = "https://www.arcgis.com"
username = ""
password = ""
survey_item_id = "" # get from url for survey, i.e. if url is "http://survey123.arcgis.com/surveys/3d02eec41a224902ab4c44e6d39319c8", then survey_item_id is 3d02eec41a224902ab4c44e6d39319c8
save_path = r"C:\temp\download_with_attachments"
keep_org_item = False
store_csv_w_attachments = False

A connection is made to your ArcGIS organization and to the survey specified above.

In [3]:
gis = GIS(portalURL, username, password)
survey_by_id = gis.content.get(survey_item_id)

Next, the data in the survey's feature service is downloaded in XLSX format. 

In [4]:
rel_fs = survey_by_id.related_items('Survey2Data','forward')[0]
item_excel = rel_fs.export(title=survey_by_id.title, export_format='Excel')
item_excel.download(save_path=save_path)
if not bool(keep_org_item):
    item_excel.delete(force=True)

Now that the data is downloaded, it's time to work with the attachments. Each layer and table in the feature service is looped through, and if the layer or table has attachments a new folder is created using the layer name with `_attachments` appended to it. A new `.csv` file is created, containing the parent object ID and the relative folder path for each attachment in that layer. The `.csv` file is saved to either the directory specified in the `save_path` variable, or in the layers folder, depending on the value of `store_csv_w_attachments`.

The layer is then queried to return the object IDs. Each object ID is queried to see if it has attachments. If it does have attachments, these are downloaded to the associated folder and a new entry is written to the `.csv` file. If the object ID does not have attachments, the record is skipped. 

When the notebook finishes, all exported data and attachments will be contained in the `save_path` folder.

In [ ]:
layers = rel_fs.layers + rel_fs.tables
for i in layers:
    if i.properties.hasAttachments == True:
        feature_layer_folder = os.path.join(save_path, '{}_attachments'.format(re.sub(r'[^A-Za-z0-9]+', '', i.properties.name)))
        os.mkdir(feature_layer_folder)
        if bool(store_csv_w_attachments):
            path = os.path.join(feature_layer_folder, "{}_attachments.csv".format(i.properties.name))
        elif not bool(store_csv_w_attachments):
            path = os.path.join(save_path, "{}_attachments.csv".format(i.properties.name))
        csv_fields = ['Parent objectId', 'Attachment path']
        with open(path, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(csv_fields)
            
            feature_object_ids = i.query(where="1=1", return_ids_only=True, order_by_fields='objectid ASC')
            for j in range(len(feature_object_ids['objectIds'])):
                current_oid = feature_object_ids['objectIds'][j]
                current_oid_attachments = i.attachments.get_list(oid=current_oid)
            
                if len(current_oid_attachments) > 0:
                    for k in range(len(current_oid_attachments)):
                        attachment_id = current_oid_attachments[k]['id']
                        current_attachment_path = i.attachments.download(oid=current_oid, attachment_id=attachment_id, save_path=feature_layer_folder)
                        csvwriter.writerow([current_oid, os.path.join('{}_attachments'.format(re.sub(r'[^A-Za-z0-9]+', '', i.properties.name)), os.path.split(current_attachment_path[0])[1])])
            

# Get pandas dataframe

In [5]:
survey_manager = arcgis.apps.survey123.SurveyManager(gis)
survey_by_id = survey_manager.get(survey_item_id)
print(survey_by_id)

<Survey @ NDPL Stationary Acoustic Metadata>


In [6]:
# Download - pandas DataFrame
import pandas as pd
survey_df = survey_by_id.download('DF')
survey_df.head()

,objectid,globalid,CreationDate,Creator,EditDate,Editor,contact_name,contact_email,survey_start_time,grts_cell_id,...,nightly_low_relative_humidity_,nightly_high_relative_humidity_,nightly_low_weather_event,nightly_high_weather_event,nightly_low_windspeed_kmhr,nightly_high_windspeed_kmhr,nightly_low_cloud_cover_,nightly_high_cloud_cover_,time_zone,SHAPE
0,1,2c0db756-9180-420b-84aa-e9f549caa7cd,2022-04-11 04:31:54.160999,ted.weller_usfs,2022-05-10 16:08:34.483000,science@batcon.org,Ted Weller,Theodore.weller@usda.gov,2022-04-09 02:34:00,59306.0,...,NaN,NaN,None,None,NaN,NaN,NaN,NaN,Pacific Time,"{""x"": -123.3875823712855, ""y"": 40.940792027920..."
1,2,26c998bd-fc6b-4132-8535-52a480a1a74e,2022-04-26 15:21:42.713999,ted.weller_usfs,2022-05-10 16:08:32.015000,science@batcon.org,Ted Weller,Theodore.weller@usda.gov,2022-04-26 03:49:00,88752.0,...,NaN,NaN,None,None,NaN,NaN,NaN,NaN,Pacific Time,"{""x"": -124.09504863193806, ""y"": 40.89769385563..."
2,3,4f0ab3d7-324f-4ecb-b411-ad0fedfc8bda,2022-04-27 14:59:35.220999,ted.weller_usfs,2022-05-10 16:08:29.088999,science@batcon.org,Ted Weller,Theodore.weller@usda.gov,2022-04-27 03:01:00,88752.0,...,NaN,NaN,None,None,NaN,NaN,NaN,NaN,Pacific Time,"{""x"": -124.09500732085574, ""y"": 40.89768247227..."
3,4,40a2b146-d001-4278-a873-4736d15601bf,2022-05-09 20:20:38.960000,,2022-05-09 20:20:38.960000,,Sharon lashway,Slashway@azgfd.gov,2022-05-03 01:54:00,6486.0,...,NaN,NaN,None,None,NaN,NaN,NaN,NaN,Mountain Time,"{""x"": -111.24162588109732, ""y"": 33.74726377323..."
4,5,c5db0db8-6762-4f66-9ee8-5fed6ca672f3,2022-05-09 20:21:37.770999,,2022-05-09 20:21:37.770999,,Sharon Lashway,Slashway@azgfd.gov,2022-05-03 01:54:00,6486.0,...,NaN,NaN,None,None,NaN,NaN,NaN,NaN,Mountain Time,"{""x"": -111.2551756813313, ""y"": 33.789939680583..."


In [7]:
survey_df.columns

Index(['objectid', 'globalid', 'CreationDate', 'Creator', 'EditDate', 'Editor',
       'contact_name', 'contact_email', 'survey_start_time', 'grts_cell_id',
       'location_name', 'land_unit_code', 'broad_habitat_type',
       'distance_to_nearest_clutter_met', 'clutter_type', 'percent_clutter',
       'distance_to_nearest_water_meter', 'water_type', 'detector',
       'detector_serial_number', 'microphone', 'microphone_serial_number',
       'microphone_orientation', 'microphone_height_meters',
       'weather_proofing', 'unusual_occurrences', 'survey_end_time',
       'nightly_low_temperature', 'nightly_high_temperature',
       'nightly_low_relative_humidity_', 'nightly_high_relative_humidity_',
       'nightly_low_weather_event', 'nightly_high_weather_event',
       'nightly_low_windspeed_kmhr', 'nightly_high_windspeed_kmhr',
       'nightly_low_cloud_cover_', 'nightly_high_cloud_cover_', 'time_zone',
       'SHAPE'],
      dtype='object')

In [8]:
len(survey_df)

120